# Demo of the Two Population Model

**Note**: the model is still under development. Results should not yet be trusted, please discuss them with me first to see what possible problems there might be. Also note that the [Binder](http://mybinder.org/) environment is temorary and requires a internet connection. Your work will definitely be lost unless you either download it, or even better, you clone the [github repository](https://github.com/birnstiel/twopoppy-demo) on your machine and run it there.

General imports and setup

In [ ]:
import twopoppy

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='svg'
from twopoppy.const import M_sun, R_sun, year, AU
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import style
style.use(['seaborn-dark',{'axes.grid': True,'font.size':14,'figure.figsize':(10,8),'lines.linewidth':2.0,}]);

Setting simulation parameters

In [ ]:
args = twopoppy.args()
args.rstar = 2.5*R_sun
args.mstar = 0.8*M_sun
args.mdisk = 0.01*args.mstar
args.rc    = 60*AU
args.gamma = 0.75
args.tstar = 3923.0
args.tmax  = 2e6*year
args.alpha = 1e-3

Running the simulation

In [ ]:
res = twopoppy.model_wrapper(args)

Assign variable names.

In [ ]:
x      = res.x
t      = res.timesteps
sig_d  = res.sigma_d
sig_da = res.sig_sol
sig_g  = res.sigma_g
a      = res.a

# Plot Evolution of the masses and dust-to-gas ratios

In [ ]:
m_d = np.trapz(2*np.pi*x*sig_d,x=x)
m_g = np.trapz(2*np.pi*x*sig_g,x=x)
f,ax = plt.subplots(2,1,figsize=(10,8))
ax[0].loglog(t/year,m_d/m_g,label='dust-to-gas')
ax[0].set_xlabel('time [yr]')
ax[0].set_ylabel('global dust-to-gas ratio')
ax[0].set_ylim([2e-4,2e-2])

ax[1].loglog(t/year,m_g/M_sun,label='gas')
ax[1].loglog(t/year,100*m_d/M_sun,label='dust$\\times 100$')
ax[1].set_xlabel('time [yr]')
ax[1].set_ylabel('Mass [$M_\odot$]')
ax[1].legend(loc='best')
ax[1].set_ylim([2e-5,2e-1])

ax[0].set_title('Evolution of the global dust-to-gas ratio and dust/gas masses');

## Evolution of the gas accretion rate

In [ ]:
f,ax=plt.subplots()
ax.loglog(t[1:]/year,-np.diff(m_g)/np.diff(t)/M_sun*year)
ax.set_xlabel('time [yr]')
ax.set_ylabel('$\dot M_\mathrm{g}$ [$M_\odot$/yr]')
ax.set_title('Evolution of the gas accretion rate');

## Snapshots of the radial dust-to-gas profiles

In [ ]:
f,ax=plt.subplots()
for _t in [0.1,0.2,0.5]:
    it = abs(t-_t*1e6*year).argmin()
    ax.loglog(x/AU,sig_d[it]/sig_g[it]*(sig_d[it]>1e-9),label='{} Myr'.format(_t))
ax.set_xlabel('r [AU]')
ax.set_ylabel(r'$\Sigma_\mathrm{d}/\Sigma_\mathrm{g}$')
ax.set_xlim(1e0,5e2)
ax.set_ylim(1e-5,1e-1)

ax.legend(fontsize='small');

## Snapshots of the surface densities

In [ ]:
f,ax=plt.subplots()
for _t in [0.1,0.2,0.5]:
    it = abs(t-_t*1e6*year).argmin()
    l,=ax.loglog(x/AU,sig_g[it]/100.,label='{} Myr'.format(_t))
    ax.loglog(x/AU,sig_d[it],c=l.get_color(),ls='--')
ax.set_xlabel('r [AU]')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.set_xlim(1e0,1e3)
ax.set_ylim(1e-5,1e3)

ax.plot(1e-100,1e-100,'k-',label='$\Sigma_\mathrm{g}$/100')
ax.plot(1e-100,1e-100,'k--',label='$\Sigma_\mathrm{d}$')
ax.legend(fontsize='small')
f.savefig('1.pdf');

In [ ]:
a0=0.01
a1=0.1

ia0 = abs(a-a0).argmin()
ia1 = abs(a-a1).argmin()

f,ax=plt.subplots()
ax.loglog(x/AU,sig_g[it]/100.,            label='$\Sigma_\mathrm{g}$/100')
ax.loglog(x/AU,sig_da[ia0:ia1+1,:].sum(0),label='$\Sigma_\mathrm{{d}}$ [{:} mm -{:} mm]'.format(10*a0,10*a1))
ax.loglog(x/AU,sig_d[it],                 label='$\Sigma_\mathrm{d,total}$')
ax.set_xlim(1e0,5e2)
ax.set_ylim(1e-6,1e2)

ax.set_xlabel('r [AU]')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.legend(loc='best');
f.savefig('2.pdf')

Get three size distributions by calling the distribution reconstruction directly.

In [ ]:
a0=0.01
a1=1.0

ia0 = np.abs(a-a0).argmin()
ia1 = np.abs(a-a1).argmin()

f,ax=plt.subplots()
for _t in [0.1,0.2,0.5]:
    it = abs(res.timesteps-_t*1e6*year).argmin()
    sig_sol,_,_,_,_,_ = twopoppy.distribution_reconstruction.reconstruct_size_distribution(res.x, res.a, res.timesteps[it], res.sigma_g[it], res.sigma_d[it], res.args.alpha*np.ones(res.args.nr), res.args.rhos, res.T, res.args.mstar, res.args.vfrag, a_0=res.args.a0)
    l,=ax.loglog(res.x/AU,res.sigma_g[it]/100.,label='{} Myr'.format(_t))
    ax.loglog(res.x/AU,sig_sol[ia0:ia1+1,:].sum(0),ls='--',c=l.get_color())
    
ax.set_xlim(1e0,5e2)
ax.set_ylim(1e-7,1e0)

ax.set_xlabel('r [AU]')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.set_xlim(1e0,5e2)
ax.set_ylim(1e-7,1e0)

ax.plot(1e-100,1e-100,'k-',label='$\Sigma_\mathrm{g}$/100')
ax.plot(1e-100,1e-100,'k--',label='$\Sigma_\mathrm{{d}}$ [{:} mm -{:} mm]'.format(10*a0,10*a1))
ax.legend(fontsize='small')
f.savefig('3.pdf');

## Plot reconstructed particle size distribution

In [ ]:
from matplotlib.colors import LogNorm
from matplotlib import ticker
f,ax = plt.subplots()

# contour plot and axes layout
c=ax.contourf(res.x/AU,res.a,sig_sol,np.logspace(-10,1,11),cmap='viridis',norm=LogNorm())
ax.set_xlim(1e-1,5e2)
ax.set_ylim(1e-4,2e2)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('r [AU]')
ax.set_ylabel('particle size [cm]')

# color bar

cb=plt.colorbar(c,ax=ax)

cb.ax.set_axis_bgcolor('none')
cb.solids.set_edgecolor('none')
cb.solids.set_linewidth(0)
cb.solids.set_antialiased(True)
cb.patch.set_visible(False)
cb.locator = ticker.LogLocator(numticks=7)
cb.set_label('$\log (\Sigma_\mathrm{d}(r,a)\cdot a ) $ [g cm$^{-2}$]')
cb.update_ticks()

To present the results as slides, save the notebook with the figures and then execute the cell below. Make sure to interrupt the kernel (via the "stop" button or the menu) when you are done. **Note:** This does not work on [Binder](http://www.mybinder.org), but only locally.

In [ ]:
!jupyter nbconvert demo.ipynb --to slides --post serve --template output_toggle